In [35]:
from bs4 import BeautifulSoup
import csv
import os
import datetime as dt 
from splinter import Browser
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import json
import pandas as pd
import datetime as dt
import numpy 

In [2]:
# Web Scraped Data: 
# [KVUE Allergy Calendar](https://kvue.com/allergy)
# Scrape Monthly Allergy Calendar data as PDFs 
# save each to html
# loop through html files to make a list 

files = os.listdir('resources/html_files/cal_v2')
url = 'resources/html_files/cal_v2'

file_list = []
for f in files:
    file_list.append(url + "/" + f)  
file_list

['resources/html_files/cal_v2/09_sep.html',
 'resources/html_files/cal_v2/04_apr.html',
 'resources/html_files/cal_v2/08_aug.html',
 'resources/html_files/cal_v2/10_oct.html',
 'resources/html_files/cal_v2/07_jul.html',
 'resources/html_files/cal_v2/01_jan.html',
 'resources/html_files/cal_v2/06_jun.html',
 'resources/html_files/cal_v2/05_may.html',
 'resources/html_files/cal_v2/03_mar.html',
 'resources/html_files/cal_v2/02_feb.html']

In [3]:
# create list to put all the data into
raw_data = []

In [4]:
# loop through each html file from KVUE to scrap
for file in file_list:
    print(f'---- Processing file {file} -----')
    
    # scrape file
    soup = BeautifulSoup(open(file), "lxml")
    
    # get month & date
    month_date = soup.find('p', class_='s2').text
    # print(month_date)
    
    # get all the days (which are td's in the table)
    days = soup.find('table').find_all('td')
    
    #  loop through all the days and gather info for the month
    counter = 0
    calc_day = 0
    
    for day in days:
        try:
            data = day.find_all('p')

            # check to make sure we have two data elements. if two elements, we know it's actual day
            # even if the day element is empty/missing
            if len(data) > 1:
                # need a date calculator as bad html doesn't always produce a day number in the td
                calc_day +=1

                day = data[0].text
                allergens = data[1].text

                print(f'Counter {counter} Day: {day} Calc Day: {calc_day} Allergens: {allergens}')
                raw_data_dict = {}
                raw_data_dict["MonthYear"] = month_date
                raw_data_dict["Day"] = calc_day
                raw_data_dict["Allergens"] = allergens
                raw_data.append(raw_data_dict)
                
            else:
                # does not have two data elements. see if the day element has a value
                day = data[0].text
                if day != '':
                    # check to see if we already have this day in the dicitonary. 
                    # if so, skip it. just bad html formatting.
                    # if not, let's add it cuz it means no allergen data found for that day
                    if not any(data['MonthYear'] == month_date and data['Day'] == int(day) for data in raw_data):
                        print(f'Missing Allergen. Counter {counter} Day: {day}')
                        raw_data_dict = {}
                        raw_data_dict["MonthYear"] = month_date
                        raw_data_dict["Day"] = int(day)
                        raw_data_dict["Allergens"] = ''
                        raw_data.append(raw_data_dict)  

                        calc_day +=1
                    
        except:
            print(f'Issue for counter {counter}')
        finally:
            counter += 1

---- Processing file resources/html_files/cal_v2/09_sep.html -----
Counter 0 Day:  Calc Day: 1 Allergens: Mold Medium.
Counter 2 Day: 2 Calc Day: 2 Allergens: Mold Medium.
Counter 3 Day:  Calc Day: 3 Allergens: Molds High with Alternaria.
Counter 5 Day:  Calc Day: 4 Allergens: Mold Medium
Counter 7 Day:  Calc Day: 5 Allergens: Fall Elm Low 7 gr/m3, Grass Medium 20 gr/m3, and Mold Medium with Alternaria
Counter 9 Day:  Calc Day: 6 Allergens: Mold Medium.
Counter 11 Day:  Calc Day: 7 Allergens: Mold Medium.
Counter 13 Day:  Calc Day: 8 Allergens: Grass Medium 13 gr/m3, and Mold Medium with Alternaria
Counter 15 Day: 9 Calc Day: 9 Allergens: Ragweed medium 13 gr/m3, Mold Medium with alternaria
Counter 16 Day:  Calc Day: 10 Allergens: Ragweed Low 7 gr/m3, Mold Medium
Counter 18 Day:  Calc Day: 11 Allergens: Ragweed Low 7 gr/m3, Mold Medium
Counter 20 Day: 12 Calc Day: 12 Allergens: Ragweed Medium 27 gr/m3, Mold Medium with Alternaria.
Counter 21 Day: 13 Calc Day: 13 Allergens: Ragweed Medi

In [5]:
# export data to csv
keys = raw_data[0].keys()
# print(keys)

with open('resources/csv_files/kvue_monthly_allergens.csv', 'a') as output_file:
    dict_writer = csv.DictWriter(output_file, restval="-", fieldnames=keys, delimiter=',')
    dict_writer.writeheader()
    dict_writer.writerows(raw_data)

In [133]:
file = ('resources/csv_files/kvue_monthly_allergens.csv')
file


'resources/csv_files/kvue_monthly_allergens.csv'

In [134]:
data = pd.read_csv(file) 
data



,MonthYear,Day,Allergens
0,September 2019,1,Mold Medium.
1,September 2019,2,Mold Medium.
2,September 2019,3,Molds High with Alternaria.
3,September 2019,4,Mold Medium
4,September 2019,5,"Fall Elm Low 7 gr/m3, Grass Medium 20 gr/m3, a..."
5,September 2019,6,Mold Medium.
6,September 2019,7,Mold Medium.
7,September 2019,8,"Grass Medium 13 gr/m3, and Mold Medium with Al..."
8,September 2019,9,"Ragweed medium 13 gr/m3, Mold Medium with alte..."
9,September 2019,10,"Ragweed Low 7 gr/m3, Mold Medium"


In [135]:
# date_d =[]
# for d in data['Day']:
#     date_d.append(d)
# len(date_d)
date_my =[]
for d in data['MonthYear']:
    date_my.append(d)
len(date_my)

# dates = zip(date_d, date_my)
# date_list = list(dates)
# date_list
# date_time_str = ''
# date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')

# print('Date:', date_time_obj.date())
# print('Time:', date_time_obj.time())
# print('Date-time:', date_time_obj)

1219

In [114]:
weather = pd.read_csv("resources/owm/weather.csv")

In [62]:
weather.head()

,dt,dt_iso,city_id,city_name,lat,lon,temp,temp_min,temp_max,pressure,...,rain_today,snow_1h,snow_3h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1349096400,2012-10-01 13:00:00 +0000 UTC,4671654,NaN,NaN,NaN,288.900000,287.150000,290.370000,1012,...,NaN,NaN,NaN,NaN,NaN,1,741,Fog,fog,50d
1,1349100000,2012-10-01 14:00:00 +0000 UTC,4671654,NaN,NaN,NaN,289.015594,289.015594,289.015594,1012,...,NaN,NaN,NaN,NaN,NaN,1,800,Clear,sky is Clear,01
2,1349103600,2012-10-01 15:00:00 +0000 UTC,4671654,NaN,NaN,NaN,289.131188,289.131188,289.131188,1012,...,NaN,NaN,NaN,NaN,NaN,1,800,Clear,sky is Clear,01
3,1349107200,2012-10-01 16:00:00 +0000 UTC,4671654,NaN,NaN,NaN,289.246781,289.246781,289.246781,1012,...,NaN,NaN,NaN,NaN,NaN,1,800,Clear,sky is Clear,01
4,1349110800,2012-10-01 17:00:00 +0000 UTC,4671654,NaN,NaN,NaN,289.362406,289.362406,289.362406,1012,...,NaN,NaN,NaN,NaN,NaN,1,800,Clear,sky is Clear,01


In [63]:
columns = ['dt','city_name','city_id','lat','lon','weather_icon', 'sea_level', 'grnd_level', 
           'snow_1h', 'snow_3h', 'snow_24h', 'rain_1h', 'rain_3h', 'rain_24h',
          'weather_main', 'weather_description']

weather = weather.drop(columns, axis=1)



In [64]:
weather


,dt_iso,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_today,snow_today,clouds_all,weather_id
0,2012-10-01 13:00:00 +0000 UTC,288.900000,287.150000,290.370000,1012,93,2,290,NaN,NaN,1,741
1,2012-10-01 14:00:00 +0000 UTC,289.015594,289.015594,289.015594,1012,92,2,278,NaN,NaN,1,800
2,2012-10-01 15:00:00 +0000 UTC,289.131188,289.131188,289.131188,1012,92,2,266,NaN,NaN,1,800
3,2012-10-01 16:00:00 +0000 UTC,289.246781,289.246781,289.246781,1012,91,2,255,NaN,NaN,1,800
4,2012-10-01 17:00:00 +0000 UTC,289.362406,289.362406,289.362406,1012,91,2,243,NaN,NaN,1,800
5,2012-10-01 18:00:00 +0000 UTC,289.478000,289.478000,289.478000,1012,90,2,232,NaN,NaN,1,800
6,2012-10-01 19:00:00 +0000 UTC,289.593594,289.593594,289.593594,1012,90,2,220,NaN,NaN,1,800
7,2012-10-01 20:00:00 +0000 UTC,289.709187,289.709187,289.709187,1012,89,2,208,NaN,NaN,1,800
8,2012-10-01 21:00:00 +0000 UTC,289.824813,289.824813,289.824813,1012,89,2,197,NaN,NaN,1,800
9,2012-10-01 22:00:00 +0000 UTC,289.940406,289.940406,289.940406,1012,89,2,185,NaN,NaN,1,800


In [65]:
# Rename columns for units clarity

rename = {
        "dt_iso" : "date", 
#         "city_name" : "city", 
        "temp" : "temp_K", 
        "temp_min" : "temp_min_K",
        "temp_max" : "temp_max_K", 
        "pressure" : "pres_hPa", 
        "humidity" : "humid_%", 
        "wind_speed" : "wind_mph",
        "rain_today" : "rain_mm", 
        "snow_today" : "snow_mm",
        "clouds_all" : "cloud_%"
}

weather = weather.rename(rename, axis=1)
weather.head()

,date,temp_K,temp_min_K,temp_max_K,pres_hPa,humid_%,wind_mph,wind_deg,rain_mm,snow_mm,cloud_%,weather_id
0,2012-10-01 13:00:00 +0000 UTC,288.900000,287.150000,290.370000,1012,93,2,290,NaN,NaN,1,741
1,2012-10-01 14:00:00 +0000 UTC,289.015594,289.015594,289.015594,1012,92,2,278,NaN,NaN,1,800
2,2012-10-01 15:00:00 +0000 UTC,289.131188,289.131188,289.131188,1012,92,2,266,NaN,NaN,1,800
3,2012-10-01 16:00:00 +0000 UTC,289.246781,289.246781,289.246781,1012,91,2,255,NaN,NaN,1,800
4,2012-10-01 17:00:00 +0000 UTC,289.362406,289.362406,289.362406,1012,91,2,243,NaN,NaN,1,800


In [66]:
weather['date'] = pd.to_datetime(weather['date'], format = '%Y-%m-%d %H:%M:%S +%f %Z').dt.date
weather

,date,temp_K,temp_min_K,temp_max_K,pres_hPa,humid_%,wind_mph,wind_deg,rain_mm,snow_mm,cloud_%,weather_id
0,2012-10-01,288.900000,287.150000,290.370000,1012,93,2,290,NaN,NaN,1,741
1,2012-10-01,289.015594,289.015594,289.015594,1012,92,2,278,NaN,NaN,1,800
2,2012-10-01,289.131188,289.131188,289.131188,1012,92,2,266,NaN,NaN,1,800
3,2012-10-01,289.246781,289.246781,289.246781,1012,91,2,255,NaN,NaN,1,800
4,2012-10-01,289.362406,289.362406,289.362406,1012,91,2,243,NaN,NaN,1,800
5,2012-10-01,289.478000,289.478000,289.478000,1012,90,2,232,NaN,NaN,1,800
6,2012-10-01,289.593594,289.593594,289.593594,1012,90,2,220,NaN,NaN,1,800
7,2012-10-01,289.709187,289.709187,289.709187,1012,89,2,208,NaN,NaN,1,800
8,2012-10-01,289.824813,289.824813,289.824813,1012,89,2,197,NaN,NaN,1,800
9,2012-10-01,289.940406,289.940406,289.940406,1012,89,2,185,NaN,NaN,1,800


In [68]:
df = weather[:40]
df

,date,temp_K,temp_min_K,temp_max_K,pres_hPa,humid_%,wind_mph,wind_deg,rain_mm,snow_mm,cloud_%,weather_id
0,2012-10-01,288.900000,287.150000,290.370000,1012,93,2,290,NaN,NaN,1,741
1,2012-10-01,289.015594,289.015594,289.015594,1012,92,2,278,NaN,NaN,1,800
2,2012-10-01,289.131188,289.131188,289.131188,1012,92,2,266,NaN,NaN,1,800
3,2012-10-01,289.246781,289.246781,289.246781,1012,91,2,255,NaN,NaN,1,800
4,2012-10-01,289.362406,289.362406,289.362406,1012,91,2,243,NaN,NaN,1,800
5,2012-10-01,289.478000,289.478000,289.478000,1012,90,2,232,NaN,NaN,1,800
6,2012-10-01,289.593594,289.593594,289.593594,1012,90,2,220,NaN,NaN,1,800
7,2012-10-01,289.709187,289.709187,289.709187,1012,89,2,208,NaN,NaN,1,800
8,2012-10-01,289.824813,289.824813,289.824813,1012,89,2,197,NaN,NaN,1,800
9,2012-10-01,289.940406,289.940406,289.940406,1012,89,2,185,NaN,NaN,1,800
